In [1]:
import torch 
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets,transforms
import mlflow
import mlflow.pytorch

In [2]:
class Config:
    EPOCH = 4
    BATCH_SIZE = 64
    LR = 0.01
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    GAMMA = 0.7
    SEED = 42
    LOG_INTERVAL = 10
    TEST_BATCH_SIZE = 1000
    DRY_RUN = False

In [3]:
config = Config()

In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1,32,3,1)
        self.conv2 = nn.Conv2d(32,64,3,1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216,128)
        self.fc2 = nn.Linear(128,10)
        
    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        x = self.dropout1(x)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        output = F.log_softmax(x,dim=1)
        return output
    

In [5]:
def train(config,model,device,train_loader,optimizer,epoch):
    model.train()
    for batch_idx,batch in enumerate(train_loader):
        data, target = batch[0].to(device), batch[1].to(device)
#         print("data_length : ",len(data))
#         print("batch_idx : ",batch_idx)
#         print("trainloader : " ,len(train_loader))
        optimizer.zero_grad()
        pred = model(data)
        loss = F.cross_entropy(pred,target)
        loss.backward()
        optimizer.step()
        if batch_idx % config.LOG_INTERVAL == 0:
            print(f"train epoch: {epoch} [{batch_idx * len(data)} / {len(train_loader.dataset)} ({100*batch_idx / len(train_loader):.0f}%)]) \t Loss:  {loss.item(): .6f}")
            
            if config.DRY_RUN:
                break

In [39]:
def test(model,device,test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device),target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output,target,reduction="sum").item()
            pred = output.argmax(dim=1,keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    print(
            f"\nTest set: Average loss: {test_loss:.4f} \
                \nAccuracy: {correct}/{len(test_loader.dataset)} \
                ({100*correct/len(test_loader.dataset):.0f}%)"
            )
            
    

In [7]:
torch.manual_seed(config.SEED)

In [8]:
train_kwargs = {
    'batch_size': config.BATCH_SIZE,
}


test_kwargs = {
    'batch_size': config.BATCH_SIZE,
}

In [9]:
if config.DEVICE == 'cuda':
    cuda_kwargs = {'num_workers': 1, 'pin_memory': True, 'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

In [10]:
transforms = transforms.Compose(
    [transforms.ToTensor()]
)

In [11]:
training_Data = datasets.MNIST("../data",train=True, download=True, transform=transforms)
testing_Data = datasets.MNIST("../data",train=False, transform=transforms)

train_loader = torch.utils.data.DataLoader(training_Data,**train_kwargs)
test_loader = torch.utils.data.DataLoader(testing_Data,**test_kwargs)

In [12]:
model_1 = ConvNet().to(config.DEVICE)

In [13]:
optimizer = torch.optim.Adam(model_1.parameters(),lr = config.LR)

scheduler = StepLR(optimizer,step_size=1,gamma=config.GAMMA)

In [14]:
for epoch in range(1,config.EPOCH + 1):
    train(config,
          model_1,
          config.DEVICE,
          train_loader,
          optimizer,epoch)
    scheduler.step()

train epoch: 1 [0 / 60000 (0)]) 	 Loss:   2.304539
train epoch: 1 [640 / 60000 (1)]) 	 Loss:   1.452546
train epoch: 1 [1280 / 60000 (2)]) 	 Loss:   1.114504
train epoch: 1 [1920 / 60000 (3)]) 	 Loss:   0.834865
train epoch: 1 [2560 / 60000 (4)]) 	 Loss:   0.569867
train epoch: 1 [3200 / 60000 (5)]) 	 Loss:   0.674093
train epoch: 1 [3840 / 60000 (6)]) 	 Loss:   0.462537
train epoch: 1 [4480 / 60000 (7)]) 	 Loss:   0.244187
train epoch: 1 [5120 / 60000 (9)]) 	 Loss:   0.780072
train epoch: 1 [5760 / 60000 (10)]) 	 Loss:   0.545230
train epoch: 1 [6400 / 60000 (11)]) 	 Loss:   0.506471
train epoch: 1 [7040 / 60000 (12)]) 	 Loss:   0.172677
train epoch: 1 [7680 / 60000 (13)]) 	 Loss:   0.560374
train epoch: 1 [8320 / 60000 (14)]) 	 Loss:   0.286397
train epoch: 1 [8960 / 60000 (15)]) 	 Loss:   0.246132
train epoch: 1 [9600 / 60000 (16)]) 	 Loss:   0.405840
train epoch: 1 [10240 / 60000 (17)]) 	 Loss:   0.324617
train epoch: 1 [10880 / 60000 (18)]) 	 Loss:   0.605678
train epoch: 1 [11520

train epoch: 2 [34560 / 60000 (58)]) 	 Loss:   0.198966
train epoch: 2 [35200 / 60000 (59)]) 	 Loss:   0.259983
train epoch: 2 [35840 / 60000 (60)]) 	 Loss:   0.051002
train epoch: 2 [36480 / 60000 (61)]) 	 Loss:   0.095289
train epoch: 2 [37120 / 60000 (62)]) 	 Loss:   0.397282
train epoch: 2 [37760 / 60000 (63)]) 	 Loss:   0.154320
train epoch: 2 [38400 / 60000 (64)]) 	 Loss:   0.125643
train epoch: 2 [39040 / 60000 (65)]) 	 Loss:   0.037845
train epoch: 2 [39680 / 60000 (66)]) 	 Loss:   0.109871
train epoch: 2 [40320 / 60000 (67)]) 	 Loss:   0.066733
train epoch: 2 [40960 / 60000 (68)]) 	 Loss:   0.257396
train epoch: 2 [41600 / 60000 (69)]) 	 Loss:   0.136364
train epoch: 2 [42240 / 60000 (70)]) 	 Loss:   0.100644
train epoch: 2 [42880 / 60000 (71)]) 	 Loss:   0.256655
train epoch: 2 [43520 / 60000 (72)]) 	 Loss:   0.249598
train epoch: 2 [44160 / 60000 (74)]) 	 Loss:   0.069281
train epoch: 2 [44800 / 60000 (75)]) 	 Loss:   0.094468
train epoch: 2 [45440 / 60000 (76)]) 	 Loss:   0

train epoch: 4 [8960 / 60000 (15)]) 	 Loss:   0.194243
train epoch: 4 [9600 / 60000 (16)]) 	 Loss:   0.158997
train epoch: 4 [10240 / 60000 (17)]) 	 Loss:   0.336299
train epoch: 4 [10880 / 60000 (18)]) 	 Loss:   0.065592
train epoch: 4 [11520 / 60000 (19)]) 	 Loss:   0.175700
train epoch: 4 [12160 / 60000 (20)]) 	 Loss:   0.036296
train epoch: 4 [12800 / 60000 (21)]) 	 Loss:   0.106461
train epoch: 4 [13440 / 60000 (22)]) 	 Loss:   0.050988
train epoch: 4 [14080 / 60000 (23)]) 	 Loss:   0.122861
train epoch: 4 [14720 / 60000 (25)]) 	 Loss:   0.141286
train epoch: 4 [15360 / 60000 (26)]) 	 Loss:   0.087958
train epoch: 4 [16000 / 60000 (27)]) 	 Loss:   0.218401
train epoch: 4 [16640 / 60000 (28)]) 	 Loss:   0.174844
train epoch: 4 [17280 / 60000 (29)]) 	 Loss:   0.028284
train epoch: 4 [17920 / 60000 (30)]) 	 Loss:   0.092015
train epoch: 4 [18560 / 60000 (31)]) 	 Loss:   0.060638
train epoch: 4 [19200 / 60000 (32)]) 	 Loss:   0.244641
train epoch: 4 [19840 / 60000 (33)]) 	 Loss:   0.1

In [40]:
test(model_1,Config.DEVICE,test_loader)


Test set: Average loss: 0.0528                 
Accuracy: 9838/10000                     
(98%)


In [33]:
with mlflow.start_run() as run:
    mlflow.pytorch.log_model(model_1,'model')
    model_path = mlflow.get_artifact_uri("model")
    loaded_torch_model = mlflow.pytorch.load_model(model_path)
    loaded_torch_model.eval()
    with torch.no_grad():
        test_datapoints,test_target = next(iter(test_loader))
        pred = loaded_torch_model(test_datapoints[1].unsqueeze(0).to(config.DEVICE))
        actual = test_target[1].item()
        predicted = torch.argmax(pred).item()
        print(f"actual : {actual},predicted : {predicted}")

actual : 2,predicted : 2


In [15]:
data=[10,1203,32,4,2,4,2,23,534,43]

In [18]:
for i, v in enumerate(iter(data)):
    print(i,v)

0 10
1 1203
2 32
3 4
4 2
5 4
6 2
7 23
8 534
9 43
